# Importing Libraries

In [1]:
import torch
from torch import nn, optim
import torchvision

from torchvision import datasets
import torchvision.transforms as transforms

# Loading Data 

In [3]:
transform = transforms.Compose([
    transforms.Resize(227),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

train = datasets.CIFAR10('data', train=True, download=True, transform=transform)
test = datasets.CIFAR10('data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train, batch_size=64,
    num_workers=2)
test_loader = torch.utils.data.DataLoader(test, batch_size=64, 
    num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train[0]

# MODEL

In [ ]:
class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(3, 96, 11, 4), # in_channels, out_channels, kernel_size, stride, padding
            nn.ReLU(),
            nn.MaxPool2d(3, 2), # kernel_size, stride
              
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),

            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU(),
            
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
        self.forward_prop = nn.Sequential(
            nn.Linear(256*6*6, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10),
        )

    def forward(self, X):
        input_x = self.block(X)
        output = self.forward_prop(input_x.view(X.shape[0], -1))
        return output

# Training Data

In [ ]:
model = AlexNet()
optimizer = optim.SGD(model.parameters(), lr=0.1)
loss = nn.CrossEntropyLoss()


def train(model, train_loader, optimizer, loss, epochs):

    for epoch in range(epochs):
        train_loss, n = 0, 0
        for X, y in train_loader:
            pred = model(X)
            l = loss(pred, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_loss += l.item()*X.size(0)
            n += 1
            
        train_loss /= n
        print(f"[================= epoch: {epoch}===================]")
        print(f" Training Loss: {train_loss:.3f}")

train(model, train_loader, optimizer, loss, 10)

# Testset Accuracy

In [ ]:
test_loss, test_acc, m = 0, 0, 0


for X, y in test_loader:
    pred = model(X)
    l = loss(pred, y)
    test_loss += l.item()*X.size(0)
    m += 1
    test_acc += ((nn.functional.softmax(pred)).argmax(1) == y).type(torch.float).sum().item()

test_acc /= m
test_loss /= m

print(f" Testing Loss: {test_loss:.3f}")
print(f" Testing Accuracy: {test_acc:.3f}")